In [ ]:
# Install necessary libraries
!pip install torch torchvision torchaudio
!pip install diffusers transformers
!pip install Pillow

# Import required libraries
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import os

# If you are using Google Colab, you might need to mount Google Drive to save images
from google.colab import drive
drive.mount('/content/drive')

# Load the Stable Diffusion pipeline once at the start
print("Loading Stable Diffusion model...")
pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base")
pipeline = pipeline.to("cuda" if torch.cuda.is_available() else "cpu")
print("Model loaded successfully.")

# Function to generate an image from a prompt
def generate_image(description: str, file_location: str):
    try:
        # Ensure the directory exists or create it
        if not os.path.exists(file_location):
            os.makedirs(file_location)
        
        # Generate the image using the provided description
        print(f"Generating image for prompt: {description}")
        image = pipeline(description).images[0]
        
        # Define the file path for saving the image
        image_path = os.path.join(file_location, "generated_clothing_image.png")
        
        # Save the generated image
        image.save(image_path)
        print(f"Image saved at: {image_path}")
        
        # Show the generated image
        image.show()
        
        return image_path
    
    except Exception as e:
        print(f"Error generating image: {str(e)}")
        return None

# Example usage:
# Set your description and file location (for instance, a folder in your Google Drive)
description = input("Enter discription to make image") #= "a red dress with floral patterns"
file_location = '/content/drive/MyDrive/GeneratedImages'  # Change this to your folder in Google Drive

# Generate the image and save it to the specified location
image_path = generate_image(description, file_location)

# Output the path where the image is saved
print(f"Generated image saved at: {image_path}")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


EntryNotFoundError: 404 Client Error. (Request ID: Root=1-67ce6da5-375f7de1586eb7ea0afbfab0;5a95e88a-d831-4718-9ed1-eb6e4336d56d)

Entry Not Found for url: https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/model_index.json.

In [2]:
pip install FastAPI


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00


In [6]:
from fastapi import FastAPI, HTTPException, File, UploadFile
from pydantic import BaseModel
from diffusers import StableDiffusionPipeline
import torch
import os
from PIL import Image
from io import BytesIO
from fastapi.responses import StreamingResponse

# Initialize FastAPI app
app = FastAPI()

# Load the Stable Diffusion pipeline once at the start
pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base").to("cuda")

# Define a Pydantic model to receive input data (clothing description)
class Prompt(BaseModel):
    description: str  # Prompt description from the user
    file_location: str  # User-specified file location to save the generated image

# Route to generate an image from the input prompt and handle file uploads
@app.post("/generate-image/")
async def generate_image(prompt: Prompt, file: UploadFile = File(None)):
    try:
        # Handle file upload if provided
        if file:
            # Save the uploaded file to the specified location
            file_location = os.path.join(prompt.file_location, file.filename)
            with open(file_location, "wb") as f:
                f.write(await file.read())
            print(f"File saved at: {file_location}")

        # Generate the image using the provided prompt
        print(f"Generating image for prompt: {prompt.description}")
        image = pipeline(prompt.description).images[0]

        # Ensure the user-specified directory exists
        output_dir = prompt.file_location
        os.makedirs(output_dir, exist_ok=True)

        # Define the file path for saving the image
        image_path = os.path.join(output_dir, "generated_clothing_image.png")

        # Save the generated image to the specified location
        image.save(image_path)

        # Convert the generated image to a byte stream for return
        img_byte_arr = BytesIO()
        image.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)

        # Return the image as a streaming response
        return StreamingResponse(img_byte_arr, media_type="image/png")

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error generating image: {str(e)}")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

ERROR:fastapi:Form data requires "python-multipart" to be installed. 
You can install "python-multipart" with: 

pip install python-multipart



RuntimeError: Form data requires "python-multipart" to be installed. 
You can install "python-multipart" with: 

pip install python-multipart


In [7]:
pip install python-multipart


In [8]:
from fastapi import FastAPI, HTTPException, File, UploadFile
from pydantic import BaseModel
from diffusers import StableDiffusionPipeline
import torch
import os
from PIL import Image
from io import BytesIO
from fastapi.responses import StreamingResponse

# Initialize FastAPI app
app = FastAPI()

# Load the Stable Diffusion pipeline once at the start
pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base").to("cuda")

# Define a Pydantic model to receive input data (clothing description)
class Prompt(BaseModel):
    description: str  # Prompt description from the user
    file_location: str  # User-specified file location to save the generated image

# Route to generate an image from the input prompt and handle file uploads
@app.post("/generate-image/")
async def generate_image(prompt: Prompt, file: UploadFile = File(None)):
    try:
        # Handle file upload if provided
        if file:
            # Save the uploaded file to the specified location
            file_location = os.path.join(prompt.file_location, file.filename)
            with open(file_location, "wb") as f:
                f.write(await file.read())
            print(f"File saved at: {file_location}")

        # Generate the image using the provided prompt
        print(f"Generating image for prompt: {prompt.description}")
        image = pipeline(prompt.description).images[0]

        # Ensure the user-specified directory exists
        output_dir = prompt.file_location
        os.makedirs(output_dir, exist_ok=True)

        # Define the file path for saving the image
        image_path = os.path.join(output_dir, "generated_clothing_image.png")

        # Save the generated image to the specified location
        image.save(image_path)

        # Convert the generated image to a byte stream for return
        img_byte_arr = BytesIO()
        image.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)

        # Return the image as a streaming response
        return StreamingResponse(img_byte_arr, media_type="image/png")

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error generating image: {str(e)}")



Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [9]:

pip install fastapi uvicorn diffusers torch transformers Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from fastapi import FastAPI, HTTPException, File, UploadFile
from pydantic import BaseModel
from diffusers import StableDiffusionPipeline
import torch
import os
from PIL import Image
from io import BytesIO
from fastapi.responses import StreamingResponse

# Initialize FastAPI app
app = FastAPI()

# Load the Stable Diffusion pipeline once at the start
pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base").to("cuda")

# Define a Pydantic model to receive input data (clothing description)
class Prompt(BaseModel):
    description: str  # Prompt description from the user
    file_location: str  # User-specified file location to save the generated image

# Route to generate an image from the input prompt and handle file uploads
@app.post("/generate-image/")
async def generate_image(prompt: Prompt, file: UploadFile = File(None)):
    try:
        # Handle file upload if provided
        if file:
            # Save the uploaded file to the specified location
            file_location = os.path.join(prompt.file_location, file.filename)
            with open(file_location, "wb") as f:
                f.write(await file.read())
            print(f"File saved at: {file_location}")

        # Generate the image using the provided prompt
        print(f"Generating image for prompt: {prompt.description}")
        image = pipeline(prompt.description).images[0]

        # Ensure the user-specified directory exists
        output_dir = prompt.file_location
        os.makedirs(output_dir, exist_ok=True)

        # Define the file path for saving the image
        image_path = os.path.join(output_dir, "generated_clothing_image.png")

        # Save the generated image to the specified location
        image.save(image_path)

        # Convert the generated image to a byte stream for return
        img_byte_arr = BytesIO()
        image.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)

        # Return the image as a streaming response
        return StreamingResponse(img_byte_arr, media_type="image/png")

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error generating image: {str(e)}")



In [ ]:
# Install necessary libraries
!pip install torch torchvision torchaudio
!pip install diffusers transformers
!pip install Pillow

# Import required libraries
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import os

# If you are using Google Colab, you might need to mount Google Drive to save images
from google.colab import drive
drive.mount('/content/drive')

# Load the Stable Diffusion pipeline once at the start
print("Loading Stable Diffusion model...")
pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base")
pipeline = pipeline.to("cuda" if torch.cuda.is_available() else "cpu")
print("Model loaded successfully.")

# Function to generate an image from a prompt
def generate_image(description: str, file_location: str):
    try:
        # Ensure the directory exists or create it
        if not os.path.exists(file_location):
            os.makedirs(file_location)

        # Generate the image using the provided description
        print(f"Generating image for prompt: {description}")
        image = pipeline(description).images[0]

        # Define the file path for saving the image
        image_path = os.path.join(file_location, "generated_clothing_image.png")

        # Save the generated image
        image.save(image_path)
        print(f"Image saved at: {image_path}")

        # Show the generated image
        image.show()

        return image_path

    except Exception as e:
        print(f"Error generating image: {str(e)}")
        return None

# Example usage:
# Set your description and file location (for instance, a folder in your Google Drive)
description = "a red dress with floral patterns"
file_location = '/content/drive/MyDrive/GeneratedImages'  # Change this to your folder in Google Drive

# Generate the image and save it to the specified location
image_path = generate_image(description, file_location)

# Output the path where the image is saved
print(f"Generated image saved at: {image_path}")


Mounted at /content/drive
Loading Stable Diffusion model...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Model loaded successfully.
Generating image for prompt: a red dress with floral patterns


  0%|          | 0/50 [00:00<?, ?it/s]